In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import time
import os
import json
from base64 import b64decode
from io import BytesIO
from PIL import Image
from multiprocessing import Pool
from IPython.display import display
import gc
import pickle
from tensorflow.keras.utils import pad_sequences
import re
from tensorflow.keras.utils import to_categorical

2023-02-27 18:47:24.842282: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-27 18:47:25.849377: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /hpc/mp/spack/opt/spack/linux-ubuntu20.04-zen2/gcc-10.3.0/cudnn-8.2.4.15-11.4-eluwegpwn6adr7hlku5p5wru5xzefpop/lib64:/hpc/mp/spack/opt/spack/linux-ubuntu20.04-zen2/gcc-10.3.0/cuda-11.4.4-ctldo35wmmwws3jbgwkgjjcjawddu3qz/lib64:/hpc/mp/spack/opt/spack/linux-ubuntu20.04-zen2/gcc-10.3.0/neovim-0.7.0-terkir3wk5rst6ktv4uxyaqjditacv5p/lib
2023-02-27 18:47:25.849448: W tensorflow/compiler/xla/stream_execut

In [15]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# eager execution
tf.compat.v1.enable_eager_execution()

### Constants

In [3]:
DATA_FOLDER = '/work/users/skoka/Data/WikipediaImageCaptionsProccessed/'
PICKLE_DIRECTORY = "/work/users/skoka/Data/WikipediaImageCaptionsPickles/"
TARGET_LANGUAGES = ['en', 'es', 'fr']

In [4]:
# lists all .json files in the data folder
files = [f for f in os.listdir(DATA_FOLDER) if f.endswith('.json')]
files.sort()
pickles = [f for f in os.listdir(PICKLE_DIRECTORY + "processed_data") if f.endswith('.pickle')]
pickles.sort()

In [5]:
# each json file has multiple json documents so we need to split them before loading
def load_file_into_json(file):
    with open(file) as f:
        json_list = [json.loads(line) for line in f.read().splitlines()]
    return json_list

### Processing Data into the format shown below:

[
    {
        "image": "numpy array of image", 
        "captions": {
            "en" : "the caption",
            "es" : "el caption",
            ....
        }
    }
]


This code will then dump all the data into pickle files for later usage.

In [6]:
def _image_bytes_to_numpy(image_bytes):
    image_decoded = b64decode(image_bytes)
    image = Image.open(BytesIO(image_decoded)).convert("RGB") # Opens image from the decoded bytes
    image = image.resize((250, 250)) # Resizes all images to 250x250 to remain consistent
    image = np.array(image) # Converts image to numpy array
    return image

# Gets the captions for the targeted languages if they exist
def _process_captions(captions):
    processed_captions = {}
    for caption in captions:
        if caption['language'] in TARGET_LANGUAGES:
            # Sometimes the caption is non existent so we need to catch the error
            try:
                processed_captions[caption['language']] = caption['caption_reference_description']
            except:
                continue
    return processed_captions

def process_data(json_list):
    data = []
    for json_doc in json_list:
        image = _image_bytes_to_numpy(json_doc['b64_bytes'])
        captions = _process_captions(json_doc['wit_features'])
        curr_data = {
            'image': image,
            'captions': captions
        }
        data.append(curr_data)
    return data
def load_data_from_pickle(pickle_file):
    with open(PICKLE_DIRECTORY + "processed_data/" + pickle_file, 'rb') as f:
        data = pickle.load(f)
    return data

### Processing Data to use in English only caption generation:

In [7]:
from keras.preprocessing.text import Tokenizer
with open(PICKLE_DIRECTORY + "captions.pickle", "rb") as f:
    captions = pickle.load(f)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(captions)
vocab_size = len(tokenizer.word_index) + 1
tokenizer.word_index['<STAAART>'] = vocab_size
tokenizer.word_index['<ENDDD>'] = vocab_size + 1
vocab_size += 2
word_index = tokenizer.word_index

In [9]:
def load_embeddings(filename, embed_size):
    # the embed size should match the file you load glove from
    embeddings_index = {}
    f = open(filename)
    # save key/array pairs of the embeddings
    #  the key of the dictionary is the word, the array is the embedding
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

    print('Found %s word vectors.' % len(embeddings_index))

    # now fill in the matrix, using the ordering from the
    #  keras word tokenizer from before
    found_words = 0
    embedding_matrix = np.zeros((len(word_index) + 1, embed_size))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be ALL-ZEROS
            embedding_matrix[i] = embedding_vector
            found_words = found_words+1

    print("Embedding Shape:",embedding_matrix.shape, "\n",
        "Total words found:",found_words, "\n",
        "Percentage:",100*found_words/embedding_matrix.shape[0])
    return embedding_matrix

embedding_matrix = load_embeddings("/users/skoka/Documents/ML-Lab2-Multi-Modal/numberbatch-en-19.08.txt", 300)

Found 516783 word vectors.
Embedding Shape: (593724, 300) 
 Total words found: 117451 
 Percentage: 19.782087299822813


In [10]:
MAX_CAPTION_LENGTH = 25
# Create a data generator to load the data in batches to avoid memory issues
# Only for English captions
def clean_caption(caption):
    caption = caption.lower()
    # remove non alphanumeric characters
    caption = re.sub(r'[^a-zA-Z0-9\s]', '', caption)
    # add start and end tokens
    caption = '<STAAART> ' + caption + ' <ENDDD>'
    return caption

def data_generator(batch_size=32):
    X1, X2, y = [], [], []
    n = 0
    while True:
        for pickle_file in pickles:
            data = load_data_from_pickle(pickle_file)
            for doc in data:
                image = doc['image']
                captions = doc['captions']
                if 'en' in captions:
                    caption = captions['en']
                    caption = clean_caption(caption)
                    caption = tokenizer.texts_to_sequences([caption])[0]
                    for i in range(1, len(caption)):
                        in_seq, out_seq = caption[:i], caption[i]
                        in_seq = pad_sequences([in_seq], maxlen=MAX_CAPTION_LENGTH)[0]
                        out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                        X1.append(image)
                        X2.append(in_seq)
                        y.append(out_seq)
                        n += 1
                        if n == batch_size:
                            yield [np.array(X1), np.array(X2)], np.array(y)
                            X1, X2, y = [], [], []
                            n = 0

In [11]:
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

with strategy.scope():
    # Build a VGG model that takes input of size 250x250x3
    vgg = tf.keras.applications.VGG19(input_shape=(250, 250, 3), include_top=False, weights='imagenet')

    # remove the last layer of the VGG model
    vgg = tf.keras.Model(inputs=vgg.inputs, outputs=vgg.layers[-2].output)
    vgg.trainable = False

    image_dense = tf.keras.layers.Flatten()(vgg.output)
    image_dense = tf.keras.layers.Dense(256, activation='relu')(image_dense)

    # Build a LSTM model that takes input of size 25
    caption_input = tf.keras.layers.Input(shape=(MAX_CAPTION_LENGTH,))
    # caption_embedding = tf.keras.layers.Embedding(vocab_size, 50, mask_zero=True)(caption_input)
    caption_embedding = tf.keras.layers.Embedding(vocab_size, 300, weights=[embedding_matrix], trainable=False, input_length=MAX_CAPTION_LENGTH)(caption_input)
    caption_lstm = tf.keras.layers.LSTM(256)(caption_embedding)


    # Merge the two models
    decoder1 = tf.keras.layers.add([image_dense, caption_lstm])
    decoder2 = tf.keras.layers.Dense(256, activation='relu')(decoder1)
    outputs = tf.keras.layers.Dense(vocab_size, activation='softmax')(decoder2)

    model = tf.keras.Model(inputs=[vgg.input, caption_input], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    # plot model:
    model.summary()




INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')


2023-02-27 18:52:30.403614: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-27 18:52:34.457130: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79117 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:07:00.0, compute capability: 8.0
2023-02-27 18:52:34.461039: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 79117 MB memory:  -> device: 1, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:0f:00.0, compute capability: 8.0
2023-02-27 18:52:34.470604: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/devi

Number of devices: 8
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 250, 250, 3  0           []                               
                                )]                                                                
      

In [17]:
generator = data_generator(batch_size=128)
model.fit_generator(generator, epochs=1, steps_per_epoch=1000, verbose=1)

/tmp/ipykernel_1082989/4048201249.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator, epochs=1, steps_per_epoch=1000, verbose=1)
2023-02-27 19:36:17.577110: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorDataset/_1"
op: "TensorDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_INT32
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 1
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\021TensorDataset:129"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
    }
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      

1000/1000 [==============================] - 232s 232ms/step - loss: 11.2439


In [18]:
def predict_caption(model, image):
    in_text = '<STAAART>'
    for i in range(MAX_CAPTION_LENGTH):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=MAX_CAPTION_LENGTH)
        yhat = model.predict([image, sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = tokenizer.index_word[yhat]
        in_text += ' ' + word
        if word == '<ENDDD>':
            break
    return in_text

In [19]:
generator = data_generator(batch_size=1)
for i in range(10):
    [image, seq], _ = next(generator)
    yhat = predict_caption(model, image)
    print(yhat)

2023-02-27 19:45:42.618958: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at cudnn_rnn_ops.cc:1564 : UNKNOWN: CUDNN_STATUS_BAD_PARAM
in tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc(1662): 'cudnnSetTensorNdDescriptor( tensor_desc.get(), data_type, sizeof(dims) / sizeof(dims[0]), dims, strides)'
2023-02-27 19:45:42.619003: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at cudnn_rnn_ops.cc:1564 : UNKNOWN: CUDNN_STATUS_BAD_PARAM
in tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc(1662): 'cudnnSetTensorNdDescriptor( tensor_desc.get(), data_type, sizeof(dims) / sizeof(dims[0]), dims, strides)'
2023-02-27 19:45:42.619026: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at cudnn_rnn_ops.cc:1564 : UNKNOWN: CUDNN_STATUS_BAD_PARAM
in tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc(1662): 'cudnnSetTensorNdDescriptor( tensor_desc.get(), data_type, sizeof(dims) / sizeof(dims[0]), dims, strides)'
2023-02-27 19:45:42

CancelledError: Graph execution error:

RecvAsync is cancelled.
	 [[{{node model_1/dense_2/Softmax/_315}}]] [Op:__inference_predict_function_78447]